In [ ]:
# installations
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import get_scheduler, pipeline
from datasets import Dataset
import evaluate
import numpy as np
from google.colab import drive
from torch.optim import AdamW
from huggingface_hub import login

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# read tsv file
orientation_training = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/orientation-tr-train.tsv', sep='\t')

In [ ]:
print(len(orientation_training))
print(orientation_training['label'].value_counts())

16138
label
1    9390
0    6748
Name: count, dtype: int64


In [ ]:
# Handle class imbalance
class_1 = orientation_training[orientation_training['label'] == 1]
class_0 = orientation_training[orientation_training['label'] == 0]

In [ ]:
# Oversample the minority class
class_0_upsampled = resample(class_0,
                              replace=True,
                              n_samples=len(class_1),
                              random_state=42)

balanced_data = pd.concat([class_1, class_0_upsampled])

In [ ]:
# Train-validation split (90% train, 10% validation, stratified)
train_data, val_data = train_test_split(balanced_data, test_size=0.1, random_state=42, stratify=balanced_data['label'])

In [ ]:
# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

In [ ]:
# Load BERT model and tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenization function
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)

In [ ]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/16902 [00:00<?, ? examples/s]

Map:   0%|          | 0/1878 [00:00<?, ? examples/s]

In [ ]:
# load metrics
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred

    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)

    return {"Accuracy": acc}

In [ ]:
# Custom optimizer and scheduler
# increase weight decay to decrease loss
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.05)
num_training_steps = len(train_dataset) // 16 * 3  # Adjust based on batch size and epochs
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(num_training_steps * 0.1),
    num_training_steps=num_training_steps
)

In [ ]:
# Training arguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/part1_new_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,              # can increment
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    fp16=True,                              # Use mixed precision for faster training
    load_best_model_at_end=True,
    optim="adamw_torch",
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, lr_scheduler),
)

In [ ]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,2.279400,0.569177,0.765000
2,0.914500,0.271000,0.893000


TrainOutput(global_step=1584, training_loss=1.5026657846238878, metrics={'train_runtime': 539.1532, 'train_samples_per_second': 94.047, 'train_steps_per_second': 2.938, 'total_flos': 1.332289139920896e+16, 'train_loss': 1.5026657846238878, 'epoch': 2.9957406530998583})

In [ ]:
predictions = trainer.predict(val_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
labels = val_dataset['label']
print(classification_report(labels, preds))

              precision    recall  f1-score   support

           0       0.87      0.93      0.90       939
           1       0.92      0.86      0.89       939

    accuracy                           0.89      1878
   macro avg       0.89      0.89      0.89      1878
weighted avg       0.89      0.89      0.89      1878

